Montgomery Multiplication
=========

ref: http://koclab.cs.ucsb.edu/teaching/cs154/docx/Notes7-Montgomery.pdf

Let the modulus $n$ be a k-bit integer(i.e. $n \in [2^{k-1}, 2^k)$, and let $r$ be $2^k$. The Montgomery multiplication algorithm requires that $r$ and $n$ be relatively prime ($gcd(n, r)=1)$ This requirement is satisfed if $n$ is odd.

we first defined the $n$-residue of an integer $a<n$ as $\bar{a} = a. r (mod\ n)$. It is straightforward to show that the set

$$
{a.r \mod n | a \in [0,n-1]}
$$

is a complete residue system.

Given two $n$-residues $\bar{a}$ and $\bar{b}$, the Montgomery product is defined as the $n$-residue

$$
\bar{c} = \bar{a}.\bar{b}.r^{-1} (mod\ n)
$$

where $r^{-1}$ is the inverse of r modulo $n$, i.e, it is the number with the property $r^{-1}.r=1 (mod\ n)$

\begin{aligned}
\bar{c}&=\bar{a}.\bar{b}.r^{-1} &(mod\ n)\\
       &=a.r.b.r.r^{-1} &(mod\ n)\\
       &=c.r &(mod\ n)
\end{aligned}

In [71]:
from klefki.curves.secp256k1 import FiniteFieldSecp256k1  as F
from klefki.numbers import invmod
from math import ceil

In [72]:
n = F.P
r = 2 ** (n.bit_length() +1)
a = 12312312312123123121123123123121313131313123112312323131313131231123123

b = 12312318080776531123121231212123131313131231123123333123123123123123
assert a < n
assert b < n

a_ = a * r % n
b_ = b * r % n
r_inv = invmod(r, n)
c_ = a_ * b_ * r_inv % n

In [73]:
n.bit_length()

256

In [74]:
a*b*r % n == c_

True

In order to describe the Montgomery reduction algorithm, we need an additional quantity $n'$, which is the  integer with the property 

$$
r.r^{-1}-n.n'
$$

The integers $r^{-1}$ and $n'$ can both be computed by the extened Euclidean algorithm. 



$n'=(r.r^{-1}-1)/n$

In [75]:
n_ = (r * r_inv -1) // n

In [76]:
assert r * r_inv % n == 1

In [77]:
assert (r*r_inv - n * n_) == 1

The computation of MonPro($\bar{a}, \bar{b})$ is achieved as follows:

*function* MonPro($\bar{a}, \bar{b})$

Step 1. $t:=\bar{a}.\bar{b}$

Step 2. $m=t.n'\ mod \ r$

Step 3. $\bar{u}:=(t+m.n)/r$

Step 4. *if* $\bar{u}\ge n$ *then return* $\bar{u}-n$ *else return* $\bar{u}$

In [78]:
def mon_pro(a_, b_, r, n):
    t = a_ * b_
    m = (t * n_) % r
    u_ = (t + m * n) // r
    if u_ >= n:
         return u_ - n
    else:
        return u_

In [79]:
assert mon_pro(a_, b_, r, n) == c_

In [84]:
def mon_mul(a, b, n):
    r = 2 ** (n.bit_length() +1)
    a_ = (a * r) % n
    b_ = (b * r) % n
    u_ = mon_pro(a_, b_, r, n)
    return mon_pro(u_, 1, r, n)


In [85]:
assert mon_mul(a, b, n) == a * b % n

## Klefki.algorithms.montgomery_mul

In [14]:
import random
from time import time
from klefki.algorithms import montgomery_mul

In [15]:
from klefki.curves.secp256k1 import FiniteFieldSecp256k1  as F


In [59]:
dataset = [(random.randint(0, F.P), random.randint(0, F.P)) for i in range(1000000)]

In [60]:
def banchmark_origin():
    start = time()
    for (a, b) in dataset:
        (a * b) % F.P
    end = time()
    return end - start

In [61]:
def banchmark_klefki():
    start = time()
    for (a, b) in dataset:
        montgomery_mul(a, b, F.P)
    end = time()
    return end - start

In [62]:
banchmark_origin()

0.6651461124420166

In [63]:
banchmark_klefki()

5.469373941421509